In [3]:
import numpy as np
from sklearn.metrics import pairwise_distances
import bottleneck as bn
import pickle
import pandas as pd

In [3]:
with open('profile2id_check.pkl', 'rb') as f:
    profile2id = pickle.load(f)

with open('show2id_check.pkl', 'rb') as f:
    show2id = pickle.load(f)
    
answer= pd.read_csv('/opt/ml/input/check_te.csv')
recvae= torch.load('/opt/ml/input/code/RecVAE/recvae7.pt')
emb= recvae.decoder.weight
emb= emb.detach().cpu().numpy()

In [10]:
re_p2id = dict((v, k) for k, v in profile2id.items())
re_s2id = dict((v, k) for k, v in show2id.items())

# 유사도 구하기

In [4]:
sim= 1-pairwise_distances(emb, metric="cosine")
# 자기자신 제외
sim[sim == 1]= -np.inf
sim_item= np.argsort(-sim)[:,:10]

In [7]:
# item id 
sim_item

array([[  99,  100,  105, ...,  107,  124,  805],
       [2238,  801, 1603, ..., 2131,  836, 2222],
       [1046, 3252,  549, ...,  788,   39, 1911],
       ...,
       [2180, 6045, 4570, ..., 5256, 4241, 6068],
       [6591, 6079, 3674, ..., 5835, 6642, 6089],
       [5988, 3093, 5710, ..., 2001, 5278, 5285]])

In [11]:
sim_df= pd.DataFrame(sim_item)
for i in range(10):
    sim_df[i]=sim_df[i].apply(lambda x: re_s2id[x])
sim_df.reset_index(inplace=True)
sim_df.rename({'index':'item'}, axis=1, inplace=True)
sim_df['item']= sim_df['item'].apply(lambda x: re_s2id[x])

In [14]:
sim_df

,item,0,1,2,3,4,5,6,7,8,9
0,4643,4638,5171,8361,5459,6541,3826,4367,1544,8644,4370
1,170,2541,1479,4052,2105,2605,3717,327,724,1586,24
2,531,262,3104,60,362,2087,917,1022,2161,1032,2125
3,616,2078,1022,2087,2085,1029,2080,2096,1032,2018,1033
4,2140,1967,2193,2161,2139,3396,8974,2021,2968,327,2005
...,...,...,...,...,...,...,...,...,...,...,...
6802,3063,6981,5039,2514,6286,26750,6687,169,6592,611,2120
6803,110771,85790,2247,64114,27828,7759,2611,92243,5994,94405,90719
6804,79590,80860,4834,91978,6978,6992,86817,4190,2841,6986,36533
6805,118700,62137,83302,5049,117881,26391,48883,53906,25891,2247,3392


유사도 순으로 나열한 10개의 아이템들이 기준 아이템과 유사한 영화가 맞는지, title과 genre 정보로 간단히 알아보았습니다. 

In [15]:
raw_data= pd.read_csv('/opt/ml/input/data/train/train_ratings.csv')
title_data= pd.read_csv('/opt/ml/input/data/train/titles.tsv',sep='\t')
genre_data=  pd.read_csv('/opt/ml/input/data/train/genres.tsv',sep='\t')


In [16]:
genre_df= pd.DataFrame(genre_data.groupby('item')['genre'].apply(lambda x:[x]))
genre_df.reset_index(inplace=True)
num_answer= pd.DataFrame(raw_data.groupby('item')['user'].count()).reset_index().rename({'user':'num'}, axis=1)

In [18]:
df= pd.merge(genre_df, title_data, on= 'item', how='right')
df= pd.merge(df, num_answer, on='item', how='left')

In [19]:
item= 6702
print(df[df['item']==item])
for i in range(9):
    print(df[df.item == int(sim_df[sim_df['item']==item][i])])

      item       genre                                     title  num
3321  6702  [[Comedy]]  Dickie Roberts: Former Child Star (2003)   55
       item                              genre            title  num
6537  97057  [[Adventure, Documentary, Drama]]  Kon-Tiki (2012)   92
     item                                 genre                      title  \
563  1037  [[Action, Horror, Sci-Fi, Thriller]]  Lawnmower Man, The (1992)   

     num  
563  463  
      item       genre                           title  num
2114  3851  [[Comedy]]  I'm the One That I Want (2000)   56
      item                 genre                title  num
1649  2970  [[Adventure, Drama]]  Fitzcarraldo (1982)  420
       item             genre                      title  num
5605  62250  [[Crime, Drama]]  Gomorrah (Gomorra) (2008)  224
        item         genre                                          title  num
6641  103235  [[Thriller]]  Best Offer, The (Migliore offerta, La) (2013)   96
      item       genre 

- 위의 정보로만 봤을 때, 어느정도 유사도가 높게 나오기에 우선 해당 유사도행렬로 아이템을 추가해보겠습니다.

# 유사도 기반 데이터 추가

In [20]:
# 추천이 잘 안된 유저 아이디 불러오기
X= pd.read_csv('/opt/ml/input/recvae_lowest_user.csv')

In [21]:
low= pd.merge(X, raw_data, on='user', how='left')
low_cnt= pd.DataFrame(low.groupby('user')['item'].count())
low_cnt.reset_index(inplace=True)

In [22]:
low_cnt['item'].describe()

count    4737.000000
mean       85.563015
std        55.862981
min        22.000000
25%        54.000000
50%        67.000000
75%        96.000000
max       741.000000
Name: item, dtype: float64

In [31]:
print(len(low_cnt))
print(len(low_cnt[low_cnt['item']<=54]))

4737
1295


<시도 내용>
- 시청이력이 평균 이하인 유저들에게 학습 아이템을 추가하겠습니다. -> [결과 성능] recvae valid 기준 0.3996
- 시청이력이 중위값 이하인 유저들에게 학습 아이템을 추가하겠습니다. -> [결과 성능] recvae valid 기준 0.3999
- 25% 이하인 유저들에게 학습 아이템을 추가하겠습니다. 

In [32]:
low_user= low_cnt[low_cnt['item']<=54]['user'].values

In [105]:
add= raw_data[raw_data['user'].isin(low_user)]
add['item2']= add['item'].apply(lambda x: int(sim_df[sim_df['item']==x][8].values))
add['item3']= add['item'].apply(lambda x: int(sim_df[sim_df['item']==x][9].values))

/tmp/ipykernel_114627/663409294.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add['item2']= add['item'].apply(lambda x: int(sim_df[sim_df['item']==x][8].values))
/tmp/ipykernel_114627/663409294.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  add['item3']= add['item'].apply(lambda x: int(sim_df[sim_df['item']==x][9].values))


In [106]:
add_df= add[['user','item2']]
add_df.rename({'item2':'item'}, axis=1, inplace=True)
raw_data2= pd.concat([raw_data, add_df], axis=0)

/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [107]:
add_df= add[['user','item3']]
add_df.rename({'item3':'item'}, axis=1, inplace=True)
raw_data2= pd.concat([raw_data2, add_df], axis=0)

In [108]:
print('증가 전:', len(raw_data))
print('증가 후:', len(raw_data2))

증가 전: 5154471
증가 후: 5281757


학습 데이터로 넣어줄 train 셋에 새로운 데이터가 얼마나 들어가는지 보겠습니다.

In [110]:
train= pd.read_csv('/opt/ml/input/data/train/pro_sg/train.csv')

In [111]:
train[train['uid'].isin(low_user)]['uid'].nunique()

235

235명의 유저에게 학습 아이템을 추가해주었습니다

In [114]:
def numerize(tp, profile2id, show2id):
    uid = tp['user'].apply(lambda x: profile2id[x])
    sid = tp['item'].apply(lambda x: show2id[x])
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [115]:
add_df=pd.concat([add[['user','item2']], add[['user','item3']].rename({'item3':'item2'}, axis=1)], axis=0)
add_df.rename({'item2':'item'}, axis=1,inplace=True)

add_df= numerize(add_df, profile2id, show2id)
add_train= add_df[add_df['uid'].isin(train['uid'])]

In [117]:
len(add_train)

99488

총 99488개의 아이템을 추가하겠습니다.

In [118]:
new_train= pd.concat([train, add_train], axis=0)

In [119]:
print('증가 전:', len(train))
print('증가 후:', len(new_train))

증가 전: 4168598
증가 후: 4268086


In [120]:
new_train.to_csv('/opt/ml/input/data/train/pro_sg/train6.csv', index=False)